In [298]:
import h5py
import glob
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

In [83]:
df = pd.read_csv("../BraTS20 Training Metadata.csv")
file_name = df['slice_path'].apply(lambda path: path.split("/")[-1])
target = df['target']

mapping = []

for f, t in zip(file_name, target):
    mapping.append((f, t))

In [310]:
idx = np.random.randint(len(mapping))
image_save_path = "../images"

for i, (image_name, label) in enumerate(mapping):
    image_file = os.path.join("../data", image_name)
    with h5py.File(image_file, "r") as file:
        image = np.array(file['image'])
        mask = np.array(file['mask'])
        
        # plt.figure(figsize=(16, 6))
        # plt.subplot(1, 4, 1)
        # plt.axis('off')
        # plt.imshow(image[:, :, 0], cmap="gray")

        # plt.subplot(1, 4, 2)
        # plt.axis('off')
        # plt.imshow(image[:, :, 1], cmap="gray")

        # plt.subplot(1, 4, 3)
        # plt.axis('off')
        # plt.imshow(image[:, :, 2], cmap="gray")

        # plt.subplot(1, 4, 4)
        # plt.axis('off')
        # plt.imshow(image[:, :, 3], cmap="gray")

        # plt.show()

        slice = image[:, :, 3]
        flatten_img = slice.flatten()

        no_zero = len(flatten_img[np.where(flatten_img > 0.0)])
        volume = no_zero / (slice.shape[0] * slice.shape[1])

        # print(label, volume)
        
        if volume >= 0.20:
            for channel in range(4):
                saving_path = os.path.join(image_save_path, f"{label}\{i}_{channel}.png")
                cropped_size = 25
                cropped_img = image[cropped_size:-cropped_size, cropped_size:-cropped_size, channel]
                matplotlib.image.imsave(saving_path, cropped_img, cmap="gray")
            # break
        
        if i % 100 == 0:
            print(f"{i} done....")

0 done....
100 done....
200 done....
300 done....
400 done....
500 done....
600 done....
700 done....
800 done....
900 done....
1000 done....
1100 done....
1200 done....
1300 done....
1400 done....
1500 done....
1600 done....
1700 done....
1800 done....
1900 done....
2000 done....
2100 done....
2200 done....
2300 done....
2400 done....
2500 done....
2600 done....
2700 done....
2800 done....
2900 done....
3000 done....
3100 done....
3200 done....
3300 done....
3400 done....
3500 done....
3600 done....
3700 done....
3800 done....
3900 done....
4000 done....
4100 done....
4200 done....
4300 done....
4400 done....
4500 done....
4600 done....
4700 done....
4800 done....
4900 done....
5000 done....
5100 done....
5200 done....
5300 done....
5400 done....
5500 done....
5600 done....
5700 done....
5800 done....
5900 done....
6000 done....
6100 done....
6200 done....
6300 done....
6400 done....
6500 done....
6600 done....
6700 done....
6800 done....
6900 done....
7000 done....
7100 done....
7200